REQUIREMENTS:
- Write a monte carlo bingo simulator to validate part (c) 
running at least 100,000 bingo game simulations
- Create a 90% confidence interval to validate your results

In [2]:
import numpy as np
import random as rand
import pandas as pd
import scipy.stats 
import timeit


from prettytable import PrettyTable

In [3]:
def createBingoCard(nRows,nCols,minVal,maxVal):
    bingoCard = rand.sample( list(range(minVal,maxVal+1)), k = nRows*nCols )
    bingoCard = np.reshape( bingoCard , (nRows,nCols))
    return bingoCard

In [4]:
def createNumBalls(minVal,maxVal,nNumBalls):
    lstNumBalls = rand.sample( list(range(minVal,maxVal+1)), k = nNumBalls)
    return lstNumBalls

In [5]:
def one_game(bingoCard, lstPattern, lstNumBalls):

    patternFound = np.zeros( len(lstPattern) , dtype = bool )

    for ii,pattern in enumerate(lstPattern):

        selectedValues = bingoCard[pattern]

        allFound = np.all( [value in lstNumBalls for value in selectedValues ] )

        if allFound :
            patternFound[ii] = 1

    return patternFound 

In [6]:
###################
### Single Game ###
###################

# Configurations

# Bingo Card size
nRows = 5
nCols = 5

# Bingo Card values and Number Ball Values
minVal = 1
maxVal = 75
nNumBalls = 30

corners4 = np.array([ [1,0,0,0,1], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [1,0,0,0,1] ],dtype = bool)

stamp = np.array([ [0,0,0,1,1], 
                   [0,0,0,1,1], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0] ],dtype = bool)

lstPatterns = [corners4, stamp]

bingoCard = createBingoCard(nRows,nCols,minVal,maxVal)
lstNumBalls = createNumBalls(minVal,maxVal,nNumBalls)
patternFound = one_game(bingoCard, lstPatterns, lstNumBalls)

print('4 Corners: ', patternFound[0])
print('Stamp: ',patternFound[1])

4 Corners:  False
Stamp:  False


In [7]:
def simulation(nGames,nRows,nCols,minVal,maxVal,nNumBalls,lstPattern):

    t1 = timeit.default_timer()

    scoreKeeper = np.zeros( (nGames,len(lstPattern) ) ,dtype = bool)

    for ii in range(nGames):

        bingoCard = createBingoCard(nRows,nCols,minVal,maxVal)
        lstNumBalls = createNumBalls(minVal,maxVal,nNumBalls)

        patternsFound = one_game(bingoCard, lstPattern, lstNumBalls)

        scoreKeeper[ii,:] = patternsFound

    t2 = timeit.default_timer()

    execTime = t2 - t1

    return scoreKeeper , execTime

In [8]:
########################
### Single Simulation ###
########################


# Configurations
nGames = 100000

# Bingo Card size
nRows = 5
nCols = 5

# Bingo Card values and Number Ball Values
minVal = 1  
maxVal = 75
nNumBalls = 30

corners4 = np.array([ [1,0,0,0,1], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [1,0,0,0,1] ],dtype = bool)

stamp = np.array([ [0,0,0,1,1], 
                   [0,0,0,1,1], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0] ],dtype = bool)

lstPatterns = [corners4, stamp]

## Simulation Starts

results, totalRuntime = simulation(nGames,
                                   nRows,
                                   nCols,
                                   minVal,
                                   maxVal,
                                   nNumBalls,
                                   lstPattern = lstPatterns)


bothPatterns = np.logical_and( results[:,0] , results[:,1] )
corners4_notStamp = np.logical_and( results[:,0] , np.invert(results[:,1]) )


#######################

table = PrettyTable()
table.field_names = ['Pattern',"Games Won",'Probability (%)']
table.add_row(['4 Corners', np.sum(results[:,0]) , 100 * np.sum(results[:,0]) / nGames])
table.add_row(['Postage Stamp', np.sum(results[:,1]) , 100 * np.sum(results[:,1]) / nGames])
table.add_row(['Both Patterns', np.sum(bothPatterns) , 100 * np.sum(bothPatterns) / nGames])
table.add_row(['4 Corners & Not Stamp', np.sum(corners4_notStamp) , 100 * np.sum(corners4_notStamp) / nGames])


print('-- Summary -----------------\n')
print('Total Games: ',nGames, '\ntotalRuntime: ', totalRuntime ,'sec' )
print(table)


-- Summary -----------------

Total Games:  100000 
totalRuntime:  7.683904338000019 sec
+-----------------------+-----------+-----------------+
|        Pattern        | Games Won | Probability (%) |
+-----------------------+-----------+-----------------+
|       4 Corners       |    2241   |      2.241      |
|     Postage Stamp     |    2207   |      2.207      |
|     Both Patterns     |    117    |      0.117      |
| 4 Corners & Not Stamp |    2124   |      2.124      |
+-----------------------+-----------+-----------------+


In [9]:
# Configurations

nGames = 100000

# Bingo Card size
nRows = 5
nCols = 5

# Bingo Card values and Number Ball Values
minVal = 1  
maxVal = 75
nNumBalls = 30

corners4 = np.array([ [1,0,0,0,1], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [0,0,0,0,0], 
                      [1,0,0,0,1] ],dtype = bool)

stamp = np.array([ [0,0,0,1,1], 
                   [0,0,0,1,1], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0], 
                   [0,0,0,0,0] ],dtype = bool)

lstPatterns = [corners4, stamp]

nSimulation = 30


savedDict = { 'Patterns':[],
            'Number Pattern Found':[],
            'Probability':[],
            'Total Runtime':[]
        }


## Many Simulations
for ii in range(nSimulation):

    results, totalRuntime = simulation(nGames,
                            nRows,
                            nCols,
                            minVal,
                            maxVal,
                            nNumBalls,
                            lstPattern = lstPatterns)

    savedDict['Patterns'].append('4 Corners')
    savedDict['Number Pattern Found'].append( np.sum(results[:,0]) )
    savedDict['Probability'].append( np.sum(results[:,0]) / nGames)
    savedDict['Total Runtime'].append(totalRuntime)

    savedDict['Patterns'].append('Postage Stamp')
    savedDict['Number Pattern Found'].append( np.sum(results[:,1]) )
    savedDict['Probability'].append( np.sum(results[:,1]) / nGames)  
    savedDict['Total Runtime'].append(totalRuntime)

    bothPatterns = np.logical_and( results[:,0] , results[:,1] )
    savedDict['Patterns'].append('Both patterns')
    savedDict['Number Pattern Found'].append( np.sum(bothPatterns) )
    savedDict['Probability'].append( np.sum(bothPatterns) / nGames)   
    savedDict['Total Runtime'].append(totalRuntime)

    corners4_notStamp = np.logical_and( results[:,0] , np.invert(results[:,1]) )
    savedDict['Patterns'].append('4 Corners & Not Stamp')
    savedDict['Number Pattern Found'].append( np.sum(corners4_notStamp) )
    savedDict['Probability'].append( np.sum(corners4_notStamp) / nGames )  
    savedDict['Total Runtime'].append(totalRuntime)



# Create a Dataframe using my dictionary
dataset = pd.DataFrame(savedDict)

# Group patterns, find the mean of probability and runtime 
mean_dataset = dataset.groupby('Patterns',as_index= False)[['Patterns','Probability']].agg('mean') 

# Game Runtime is the same of all patterns. Pick one to find the total runtime 
totalRuntime = dataset[ dataset['Patterns'] == '4 Corners' ]['Total Runtime'].sum()

# Print the imformation
print('\n',mean_dataset.head(15))



# Calculating the probability mean Confidence Interval for ' 4 Corners & Not Stamp '

confidence=0.90
n = nSimulation
mean = mean_dataset[ mean_dataset['Patterns'] == '4 Corners & Not Stamp']['Probability'].to_numpy()[0]
standardDev = scipy.stats.sem( dataset[ dataset['Patterns'] == '4 Corners & Not Stamp']['Probability'])

interval = scipy.stats.norm.interval(confidence, mean, standardDev)

print("\nWith {:.0f}% Confidence, the population probability mean of the pattern '4 Corners & Not Stamp' is within this confidence interval".format(confidence*100))
print('Confidence Interval:({:.4f},{:.4f})'.format(interval[0] , interval[1] ) )

print('\nTotal Runtime: {:.4f} sec'.format(totalRuntime) )
print('Avg. simulation Runtime: {:4f} sec'.format( dataset['Total Runtime'].mean() ) )


                 Patterns  Probability
0              4 Corners     0.022440
1  4 Corners & Not Stamp     0.021434
2          Both patterns     0.001006
3          Postage Stamp     0.022404

With 90% Confidence, the population probability mean of the pattern '4 Corners & Not Stamp' is within this confidence interval
Confidence Interval:(0.0213,0.0216)

Total Runtime: 218.0098 sec
Avg. simulation Runtime: 7.266994 sec
